### Imports

In [129]:
import os
import numpy as np
import pandas as pd
import skfuzzy as fuzz
from skfuzzy import control as ctrl

### Inputs

In [130]:
radnet_input_df = pd.read_csv("../inputs/radnet_input.csv")
radnet_input_efsr_edge = radnet_input_df['efsr_edge'].values

In [131]:
inference_output_df = pd.read_csv("../outputs/inference_output.csv")
inference_output_class = inference_output_df['class'].values
inference_output_confidence = inference_output_df['confidence'].values

In [132]:
edges_input_df = pd.read_csv("../inputs/edges_input.csv")
edges_input = edges_input_df[["rssi", "etx", "delay", "busy_fraction"]].values

### Fuzzy Logic

In [133]:
conf_model = ctrl.Antecedent(np.arange(0, 1.01, 0.01), 'conf_model')
conf_efsr = ctrl.Antecedent(np.arange(0, 1.01, 0.01), 'conf_efsr')
quality = ctrl.Consequent(np.arange(0, 1.01, 0.01), 'quality')

In [134]:
conf_model['baixa'] = fuzz.trimf(conf_model.universe, [0, 0, 0.5])
conf_model['media'] = fuzz.trimf(conf_model.universe, [0, 0.5, 1])
conf_model['alta'] = fuzz.trimf(conf_model.universe, [0.5, 1, 1])

In [135]:
conf_efsr['baixa'] = fuzz.trimf(conf_efsr.universe, [0, 0, 0.5])
conf_efsr['media'] = fuzz.trimf(conf_efsr.universe, [0, 0.5, 1])
conf_efsr['alta'] = fuzz.trimf(conf_efsr.universe, [0.5, 1, 1])

In [136]:
quality['ruim'] = fuzz.trimf(quality.universe, [0, 0, 0.5])
quality['media'] = fuzz.trimf(quality.universe, [0, 0.5, 1])
quality['boa'] = fuzz.trimf(quality.universe, [0.5, 1, 1])

In [137]:
rules = [
    ctrl.Rule(conf_model['alta'] & conf_efsr['alta'], quality['boa']),
    ctrl.Rule(conf_model['alta'] & conf_efsr['media'], quality['media']),
    ctrl.Rule(conf_model['alta'] & conf_efsr['baixa'], quality['ruim']),
    ctrl.Rule(conf_model['media'] & conf_efsr['alta'], quality['boa']),
    ctrl.Rule(conf_model['media'] & conf_efsr['media'], quality['media']),
    ctrl.Rule(conf_model['media'] & conf_efsr['baixa'], quality['ruim']),
    ctrl.Rule(conf_model['baixa'] & conf_efsr['alta'], quality['boa']),
    ctrl.Rule(conf_model['baixa'] & conf_efsr['media'], quality['media']),
    ctrl.Rule(conf_model['baixa'] & conf_efsr['baixa'], quality['ruim'])
]

In [138]:
fuzzy_system = ctrl.ControlSystemSimulation(ctrl.ControlSystem(rules))

### Confidence

In [139]:
confidence_model = np.where(
    inference_output_class == 1,
    inference_output_confidence,
    1.0 - inference_output_confidence
)

In [140]:
efsr_min, efsr_max = 0.20, 0.98
confidence_efsr = np.clip((radnet_input_efsr_edge - efsr_min) / (efsr_max - efsr_min), 0, 1)

In [141]:
fused = np.zeros(len(confidence_model))

for i in range(len(confidence_model)):
    fuzzy_system.input['conf_model'] = confidence_model[i]
    fuzzy_system.input['conf_efsr'] = confidence_efsr[i]
    fuzzy_system.compute()
    fused[i] = fuzzy_system.output['quality']

### Inference

In [142]:
cls = (fused >= 0.5).astype(int)
conf = np.where(cls == 1, fused, 1.0 - fused)

In [143]:
print(f"{'RSSI':<6} {'ETX':<5} {'Delay':<6} {'Busy':<6} {'EFSR':<6} {'Classe':<8} {'Status':<8} {'Confiança':<10}")
print(f"{'-'*62}")

CLASS_NAMES = {0: "Ruim", 1: "Bom"}

for i, row in enumerate(edges_input):
    print(f"{row[0]:<6.0f} {row[1]:<5.1f} {row[2]:<6.0f} {row[3]:<6.2f} "
          f"{radnet_input_efsr_edge[i]:<6.2f} "
          f"{cls[i]:<8} {CLASS_NAMES[int(cls[i])]:<8} {conf[i]:<10.2f}")

RSSI   ETX   Delay  Busy   EFSR   Classe   Status   Confiança 
--------------------------------------------------------------
-59    1.1   17     0.20   0.33   0        Ruim     0.61      
-60    1.4   6      0.10   0.91   1        Bom      0.68      
-78    6.8   19     0.49   0.91   1        Bom      0.69      
-84    11.0  18     0.26   0.36   0        Ruim     0.58      
-64    3.2   7      0.20   0.36   0        Ruim     0.59      
-43    1.5   4      0.32   0.90   1        Bom      0.67      
-58    1.3   4      0.22   0.90   1        Bom      0.67      
-57    1.2   14     0.17   0.90   1        Bom      0.66      
-62    1.0   7      0.11   0.93   1        Bom      0.71      
-69    1.8   5      0.22   0.38   0        Ruim     0.57      
-62    3.1   7      0.34   0.32   0        Ruim     0.62      
-57    1.0   14     0.30   0.35   0        Ruim     0.59      
-49    2.6   18     0.19   0.41   0        Ruim     0.55      
-78    4.4   42     0.82   0.48   0        Ruim     0.5

### Save

In [144]:
fuzzy_output = pd.DataFrame({
    'class': cls,
    'confidence': conf
})

os.makedirs("../outputs", exist_ok=True)
fuzzy_output.to_csv("../outputs/fuzzy_output.csv", index=False)

print("Lógica Fuzzy salva em ../outputs/fuzzy_output.csv")

Lógica Fuzzy salva em ../outputs/fuzzy_output.csv
